# Process, Action, and Risk Model
Create any number of processes and risks, then plot process flow from first to last.

In [31]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from IPython.display import clear_output, display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd


In [32]:
@dataclass
class Process:
    process_id: str
    scrap_price: float
    finishing_price: float

@dataclass
class Action:
    type: str  # "detection" or "prevention"
    cost: int
    days: int

@dataclass
class Risk:
    process_id: str
    risk_id: str
    days: int = 0  # sum of all action days
    preventive_actions: List[Action] = field(default_factory=list)
    detection_actions: List[Action] = field(default_factory=list)

    @property
    def total_days(self) -> int:
        return int(sum(a.days for a in (self.preventive_actions + self.detection_actions)))


@dataclass
class Scrap:
    scrap_id: int
    cause_process_id: str        # where scrap happened
    consequence_process_id: str  # where it was found
    count: int                   # number of scraps (qty)
    scrap_type: str = "scrap"    # "scrap" | "finishing"
@dataclass
class Ppm:
    id: int            # 1..10
    units: int         # >= 1
    in_size: int       # >= 1

    @property
    def rate(self) -> float:
        return self.units / self.in_size

    @property
    def ppm(self) -> float:
        return self.rate * 1_000_000.0


In [33]:
def _make_strictly_increasing(values: List[float], name: str, min_step: float = 0.1) -> List[float]:
    """Return a strictly increasing list by minimally bumping later values if needed."""
    if not values:
        return values
    out = [float(values[0])]
    for v in values[1:]:
        v = float(v)
        if v <= out[-1]:
            v = out[-1] + max(min_step, abs(out[-1]) * 0.02)
        out.append(v)
    return out


def create_processes(
    m: int,
    scrap_prices: Optional[List[float]] = None,
    finishing_prices: Optional[List[float]] = None,
    process_prefix: str = 'P',
    default_scrap_start: float = 10.0,
    default_scrap_step: float = 2.0,
    default_finish_start: float = 5.0,
    default_finish_step: float = 2.0,
    enforce_increasing: bool = True,
) -> List[Process]:
    """Create m processes with (optionally) provided prices.

    Requirement:
      - scrap_price and finishing_price should increase as process index increases.
    """
    if m <= 0:
        raise ValueError('m must be greater than 0')

    if scrap_prices is None:
        scrap_prices = [default_scrap_start + i * default_scrap_step for i in range(m)]
    if finishing_prices is None:
        finishing_prices = [default_finish_start + i * default_finish_step for i in range(m)]

    if len(scrap_prices) != m:
        raise ValueError('scrap_prices length must match m')
    if len(finishing_prices) != m:
        raise ValueError('finishing_prices length must match m')

    if enforce_increasing:
        scrap_prices = _make_strictly_increasing(scrap_prices, 'scrap_prices')
        finishing_prices = _make_strictly_increasing(finishing_prices, 'finishing_prices')

    return [
        Process(process_id=f'{process_prefix}{i + 1}',
                scrap_price=float(scrap_prices[i]),
                finishing_price=float(finishing_prices[i]))
        for i in range(m)
    ]


def create_risks(
    n: int,
    processes: List[Process],
    preventive_actions_per_risk: int = 0,
    detection_actions_per_risk: int = 0,
    action_cost: int = 0,
    action_days: int = 0,
    risk_prefix: str = 'R'
) -> List[Risk]:
    """Create n risks and distribute them across processes (round-robin)."""
    if n < 0:
        raise ValueError('n cannot be negative')
    if n > 0 and not processes:
        raise ValueError('processes list cannot be empty when n > 0')
    if preventive_actions_per_risk < 0 or detection_actions_per_risk < 0:
        raise ValueError('actions per risk cannot be negative')

    risks: List[Risk] = []
    for i in range(n):
        process_id = processes[i % len(processes)].process_id
        preventive = [Action(type='prevention', cost=action_cost, days=action_days) for _ in range(preventive_actions_per_risk)]
        detection = [Action(type='detection', cost=action_cost, days=action_days) for _ in range(detection_actions_per_risk)]
        risks.append(
            Risk(
                process_id=process_id,
                risk_id=f'{risk_prefix}{i + 1}',
                preventive_actions=preventive,
                detection_actions=detection
            )
        )
    return risks


def create_single_risk(
    process_id: str,
    risk_id: str = 'R1',
    preventive_actions: Optional[List[Action]] = None,
    detection_actions: Optional[List[Action]] = None,
) -> Risk:
    # Default: exactly 1 preventive and 1 detection action
    if preventive_actions is None:
        preventive_actions = [Action(type='prevention', cost=1000, days=30)]
    if detection_actions is None:
        detection_actions = [Action(type='detection', cost=1000, days=30)]

    return Risk(
        process_id=process_id,
        risk_id=risk_id,
        preventive_actions=preventive_actions,
        detection_actions=detection_actions,
    )


def _ppm_row_by_id(rows: List[Ppm], pid: int) -> Ppm:
    for r in rows:
        if int(r.id) == int(pid):
            return r
    return rows[0]


def _clamp(x: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, x))


# Global variables

In [34]:
# Processes
m = 8
processes = create_processes(
    m=m,
    process_prefix='P',
    default_scrap_start=10.0,
    default_scrap_step=2.0,
    default_finish_start=5.0,
    default_finish_step=2.0,
    enforce_increasing=True,
)
process_options = [p.process_id for p in processes]
process_by_id: Dict[str, Process] = {p.process_id: p for p in processes}

# Single risk (example)
risk = create_single_risk(process_id='P3', risk_id='R1')

# Default scraps
scraps: List[Scrap] = [
    Scrap(scrap_id=1, cause_process_id='P3', consequence_process_id='P5', count=100, scrap_type='scrap'),
    Scrap(scrap_id=2, cause_process_id='P3', consequence_process_id='P7', count=30, scrap_type='scrap'),
    Scrap(scrap_id=3, cause_process_id='P3', consequence_process_id='P5', count=30, scrap_type='finishing'),
    Scrap(scrap_id=4, cause_process_id='P3', consequence_process_id='P7', count=30, scrap_type='finishing'),
]

# PPM tables (O / D)
O_ppm: List[Ppm] = [
    Ppm(id=i, units=1, in_size=(1 if i <= 1 else min(10 * (i - 1), 90)))
    for i in range(1, 11)
]
D_ppm: List[Ppm] = [
    Ppm(id=i, units=1, in_size=(1 if i >= 10 else 100 - i * 10))
    for i in range(1, 11)
]


def reduction_O_from_rates(O_rate: float, AP_O_rate: float) -> float:
    O_rate = float(O_rate)
    AP_O_rate = float(AP_O_rate)
    if O_rate <= 0.0:
        return 0.0
    return _clamp((O_rate - AP_O_rate) / O_rate, 0.0, 1.0)


def reduction_D_from_rates(D_rate: float, AP_D_rate: float) -> float:
    D_rate = float(D_rate)
    AP_D_rate = float(AP_D_rate)
    if AP_D_rate <= 0.0:
        return 0.0
    return _clamp(1.0 - (D_rate / AP_D_rate), 0.0, 1.0)


# Global reductions (updated by _render_diffs)
reduction_O = 0.0
reduction_D = 0.0


# Global UI controls/widgets
risk_process_dd = widgets.Dropdown(options=process_options, value=risk.process_id, description='Risk at:')

O_id_dd = widgets.Dropdown(options=[(str(r.id), r.id) for r in O_ppm], value=2, description='O:')
AP_O_id_dd = widgets.Dropdown(options=[(str(r.id), r.id) for r in O_ppm], value=3, description='AP_O:')

D_id_dd = widgets.Dropdown(options=[(str(r.id), r.id) for r in D_ppm], value=8, description='D:')
AP_D_id_dd = widgets.Dropdown(options=[(str(r.id), r.id) for r in D_ppm], value=9, description='AP_D:')

flow_out = widgets.Output()
diff_out = widgets.Output()

scraps_rows_box = widgets.VBox()
add_scrap_row_btn = widgets.Button(description='Add row', button_style='', icon='plus')

# Placeholders populated by UI cell
scraps_section = None
O_table_ui = None
D_table_ui = None



# Process flow

In [35]:
def plot_process_flow(processes: List[Process], risks: Optional[List[Risk]] = None) -> None:
    if not processes:
        raise ValueError('processes list cannot be empty')

    risks = risks or []
    pid_to_x = {p.process_id: i for i, p in enumerate(processes)}

    x_positions = list(range(len(processes)))
    fig, ax = plt.subplots(figsize=(max(8, len(processes) * 2.7), 3.6))


    # Draw processes (rectangles)
    from matplotlib.patches import Rectangle

    rect_w = 0.85
    rect_h = 0.55

    for x, p in zip(x_positions, processes):
        r = Rectangle((x - rect_w/2, -rect_h/2), rect_w, rect_h,
                      fill=True, alpha=0.18, linewidth=2, zorder=3)
        ax.add_patch(r)

        label = f"{p.process_id}\nScrap: {p.scrap_price:>4.1f}\nFin : {p.finishing_price:>4.1f}"
        ax.text(x, 0, label, ha='center', va='center', fontsize=10, zorder=4, fontfamily='monospace')
    # Draw arrows
    for i in range(len(processes) - 1):
        ax.annotate(
            '',
            xy=(x_positions[i + 1] - 0.28, 0),
            xytext=(x_positions[i] + 0.28, 0),
            arrowprops=dict(arrowstyle='->', lw=2, color='#1f1f1f')
        )

    # Draw risks (above their process)
    for r in risks:
        if r.process_id not in pid_to_x:
            continue
        x = pid_to_x[r.process_id]
        ax.scatter(x, 0.62, s=450, marker='D', c='#d9534f', edgecolors='#7a1f1a', linewidths=1.5, zorder=5)
        # Actions summary
        prev = r.preventive_actions[0] if r.preventive_actions else None
        det = r.detection_actions[0] if r.detection_actions else None
        if prev or det:
            lines = []
            if prev: lines.append(f'Prev: {prev.cost:>6} / {prev.days:>3}d')
            if det:  lines.append(f'Det : {det.cost:>6} / {det.days:>3}d')
            ax.annotate('\n'.join(lines), (x, 0), textcoords='offset points', xytext=(10, 10), ha='left', va='bottom', fontsize=9, fontfamily='monospace')
        ax.text(x, 0.62, r.risk_id, ha='center', va='center', fontsize=9, color='white', zorder=6)

    ax.set_title('Process Flow (First to Last) + Risk Location', fontsize=13)
    ax.set_xlim(-0.8, len(processes) - 0.2)
    ax.set_ylim(-0.85, 0.95)
    ax.axis('off')
    plt.show()


# UI logic

In [36]:
# Interactive UI: Risk + Scraps (no scenarios)

def _process_scrap_cost(pid: str) -> float:
    p = process_by_id.get(str(pid))
    return float(p.scrap_price) if p else 0.0


# Editable Scraps table (widget rows)
def _make_header_cell(text: str, width: str) -> widgets.HTML:
    return widgets.HTML(f"<div style='font-weight:600; width:{width};'>{text}</div>")


def _render_scraps_table() -> None:
    header = widgets.HBox([
        _make_header_cell('ID', '80px'),
        _make_header_cell('Cause', '120px'),
        _make_header_cell('Found at', '120px'),
        _make_header_cell('Type', '120px'),
        _make_header_cell('Count', '120px'),
        _make_header_cell('', '42px'),
    ])

    row_widgets = []

    def _on_row_change(_=None):
        _render_diffs()

    for s in scraps:
        id_w = widgets.IntText(value=int(s.scrap_id), disabled=True, layout=widgets.Layout(width='80px'))

        cause_w = widgets.Dropdown(options=process_options, value=str(s.cause_process_id), layout=widgets.Layout(width='120px'))
        found_w = widgets.Dropdown(options=process_options, value=str(s.consequence_process_id), layout=widgets.Layout(width='120px'))
        type_w = widgets.Dropdown(options=['scrap', 'finishing'], value=str(getattr(s, 'scrap_type', 'scrap')), layout=widgets.Layout(width='120px'))
        count_w = widgets.IntText(value=int(s.count), layout=widgets.Layout(width='120px'))

        def _bind_updates(scrap_obj: Scrap, cw: widgets.Dropdown, fw: widgets.Dropdown, tw: widgets.Dropdown, qw: widgets.IntText):
            def _update_cause(change):
                scrap_obj.cause_process_id = str(change['new'])
                _on_row_change()

            def _update_found(change):
                scrap_obj.consequence_process_id = str(change['new'])
                _on_row_change()

            def _update_type(change):
                scrap_obj.scrap_type = str(change['new'])
                _on_row_change()

            def _update_count(change):
                try:
                    scrap_obj.count = int(change['new'])
                except Exception:
                    scrap_obj.count = 0
                _on_row_change()

            cw.observe(_update_cause, names='value')
            fw.observe(_update_found, names='value')
            tw.observe(_update_type, names='value')
            qw.observe(_update_count, names='value')

        _bind_updates(s, cause_w, found_w, type_w, count_w)

        del_w = widgets.Button(description='', icon='trash', layout=widgets.Layout(width='42px'))

        def _make_delete_handler(scrap_id: int):
            def _on_delete(_btn):
                global scraps
                scraps[:] = [x for x in scraps if int(x.scrap_id) != int(scrap_id)]
                _render_scraps_table()
                _render_diffs()

            return _on_delete

        del_w.on_click(_make_delete_handler(s.scrap_id))

        row_widgets.append(widgets.HBox([id_w, cause_w, found_w, type_w, count_w, del_w]))

    scraps_rows_box.children = [header] + row_widgets


def _on_add_scrap_row(_):
    sid = 1
    if scraps:
        sid = max(int(s.scrap_id) for s in scraps) + 1

    default_cause = str(risk_process_dd.value or risk.process_id)
    try:
        i = process_options.index(default_cause)
        default_found = process_options[min(i + 1, len(process_options) - 1)]
    except Exception:
        default_found = process_options[0]

    scraps.append(Scrap(scrap_id=sid, cause_process_id=default_cause, consequence_process_id=str(default_found), scrap_type='scrap', count=1))
    _render_scraps_table()
    _render_diffs()


if not getattr(add_scrap_row_btn, '_bound_add_handler', False):
    add_scrap_row_btn.on_click(_on_add_scrap_row)
    add_scrap_row_btn._bound_add_handler = True

scraps_section = widgets.VBox([
    widgets.HTML('<h3>Scraps</h3><div>Edit rows directly. Add/remove rows as needed.</div>'),
    widgets.HBox([add_scrap_row_btn]),
    scraps_rows_box,
])


# Editable PPM tables (O / D)
def _render_ppm_table(title: str, rows: List[Ppm]) -> widgets.VBox:
    header = widgets.HBox([
        _make_header_cell('ID', '60px'),
        _make_header_cell('Units', '80px'),
        _make_header_cell('In', '80px'),
        _make_header_cell('Rate', '140px'),
    ])

    rows_box = widgets.VBox()

    def _update_rate_label(rate_html: widgets.HTML, ppm_obj: Ppm):
        rate_html.value = f"<div style='width:140px;'>{ppm_obj.rate:.8f}</div>"

    row_widgets = []
    for r in rows:
        id_w = widgets.IntText(value=int(r.id), disabled=True, layout=widgets.Layout(width='60px'))
        units_w = widgets.IntText(value=int(r.units), layout=widgets.Layout(width='80px'))
        in_w = widgets.IntText(value=int(r.in_size), layout=widgets.Layout(width='80px'))
        rate_w = widgets.HTML(f"<div style='width:140px;'>{r.rate:.8f}</div>")

        def _bind_updates(ppm_obj: Ppm, uw: widgets.IntText, iw: widgets.IntText, rw: widgets.HTML):
            def _u(change):
                try:
                    ppm_obj.units = max(1, int(change['new']))
                except Exception:
                    ppm_obj.units = 1
                _update_rate_label(rw, ppm_obj)
                _render_diffs()

            def _i(change):
                try:
                    ppm_obj.in_size = max(1, int(change['new']))
                except Exception:
                    ppm_obj.in_size = 1
                _update_rate_label(rw, ppm_obj)
                _render_diffs()

            uw.observe(_u, names='value')
            iw.observe(_i, names='value')

        _bind_updates(r, units_w, in_w, rate_w)

        row_widgets.append(widgets.HBox([id_w, units_w, in_w, rate_w]))

    rows_box.children = [header] + row_widgets
    return widgets.VBox([widgets.HTML(f'<h3>{title}</h3>'), rows_box])


O_table_ui = _render_ppm_table('O table (Occurrence)', O_ppm)
D_table_ui = _render_ppm_table('D table (Detection)', D_ppm)


# Diff + savings calculations
def _render_diffs(_=None):
    global reduction_O, reduction_D
    # Update flow plot
    risk.process_id = str(risk_process_dd.value or risk.process_id)
    with flow_out:
        clear_output(wait=True)
        plot_process_flow(processes, risks=[risk])

    O_row = _ppm_row_by_id(O_ppm, O_id_dd.value)
    AP_O_row = _ppm_row_by_id(O_ppm, AP_O_id_dd.value)
    D_row = _ppm_row_by_id(D_ppm, D_id_dd.value)
    AP_D_row = _ppm_row_by_id(D_ppm, AP_D_id_dd.value)

    O_rate = float(O_row.rate)
    AP_O_rate = float(AP_O_row.rate)

    D_rate = float(D_row.rate)
    AP_D_rate = float(AP_D_row.rate)

    # % saved scraps from Occurrence improvement
    reduction_O = reduction_O_from_rates(O_rate, AP_O_rate)

    # Treat reduction_D as relative improvement toward AP_D
    reduction_D = reduction_D_from_rates(D_rate, AP_D_rate)

    # Baseline scraps for this risk = sum rows where Cause == selected risk process
    cause_pid = str(risk_process_dd.value or risk.process_id)
    relevant_rows = [s for s in scraps if str(s.cause_process_id) == cause_pid]
    allScraps = float(sum(max(0, int(s.count)) for s in relevant_rows))

    # Saved by Occurrence (scraps avoided entirely)
    saved_occ = allScraps * reduction_O
    saved_occ = _clamp(saved_occ, 0.0, allScraps)

    # Saved by Detection (additional scraps detected early)
    saved_det = allScraps * reduction_D
    saved_det = _clamp(saved_det, 0.0, allScraps)

    base_cost_total = 0.0
    after_cost_total = 0.0

    # Detection probabilities used in cost split
    d_base = _clamp(D_rate, 0.0, 1.0)
    d_after = _clamp(AP_D_rate, 0.0, 1.0)

    for s in relevant_rows:
        base_count = float(max(0, int(s.count)))
        found_pid = str(s.consequence_process_id)

        cost_cause = _process_scrap_cost(cause_pid)
        cost_found = _process_scrap_cost(found_pid)

        # Baseline expected cost per scrap: detected-at-cause vs found-at
        base_cost_per = d_base * cost_cause + (1.0 - d_base) * cost_found
        base_cost = base_count * base_cost_per

        # After: fewer scraps due to reduction_O
        after_count = base_count * (1.0 - reduction_O)
        after_cost_per = d_after * cost_cause + (1.0 - d_after) * cost_found
        after_cost = after_count * after_cost_per

        base_cost_total += base_cost
        after_cost_total += after_cost

    cost_saved = base_cost_total - after_cost_total

    with diff_out:
        clear_output(wait=True)
        display(widgets.HTML(
            f"<div>"
            f"<b>Risk at</b>: {cause_pid} &nbsp; | &nbsp; <b>Scrap rows</b>: {len(relevant_rows)} &nbsp; | &nbsp; <b>All scraps</b>: {allScraps:,.2f}"
            f"</div>"
            f"<hr style='margin:8px 0'/>"
            f"<div>"
            f"<b>O</b>: {O_row.units} in {O_row.in_size} (rate {O_rate:.8f}) &nbsp; → &nbsp; "
            f"<b>AP_O</b>: {AP_O_row.units} in {AP_O_row.in_size} (rate {AP_O_rate:.8f})<br/>"
            f"<b>reduction_O</b>: {reduction_O:.8f} &nbsp; | &nbsp; {reduction_O*100.0:.4f}%<br/>"
            f"</div>"
            f"<hr style='margin:8px 0'/>"
            f"<div>"
            f"<b>D</b>: {D_row.units} in {D_row.in_size} (rate {D_rate:.8f}) &nbsp; → &nbsp; "
            f"<b>AP_D</b>: {AP_D_row.units} in {AP_D_row.in_size} (rate {AP_D_rate:.8f})<br/>"
            f"<b>reduction_D</b>: {reduction_D:.8f} &nbsp; | &nbsp; {reduction_D*100.0:.4f}% (fraction of scraps additionally detected early)<br/>"
            f"</div>"
        ))


# Re-render when controls change
for w in [risk_process_dd, O_id_dd, AP_O_id_dd, D_id_dd, AP_D_id_dd]:
    if not getattr(w, '_bound_diff_handler', False):
        w.observe(_render_diffs, names='value')
        w._bound_diff_handler = True


# Layout
display(widgets.HTML('<h2>Risk + O/D Diffs</h2>'))
display(risk_process_dd)
display(flow_out)
display(widgets.HBox([O_id_dd, AP_O_id_dd, D_id_dd, AP_D_id_dd]))
display(widgets.HBox([O_table_ui, D_table_ui]))
display(scraps_section)
display(diff_out)

_render_scraps_table()
_render_diffs()



HTML(value='<h2>Risk + O/D Diffs</h2>')

Dropdown(description='Risk at:', index=2, options=('P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8'), value='P3'…

Output()

Output()

# Helpers

In [37]:
# -------------------------
# Scenario helpers: cumulative value-added over ordered processes
# Cum(Px) = price(P1)+price(P2)+...+price(Px)
# CumRange(Px, Py) = price(Px)+...+price(Py) (or exclude Px via include_from=False)
# -------------------------

def CumRange(from_pid: str, to_pid: str, scrap_type: str = "scrap", include_from: bool = True) -> float:
    from_pid = str(from_pid)
    to_pid = str(to_pid)
    if not from_pid or not to_pid:
        return 0.0

    use_finishing = str(scrap_type).lower() == "finishing"
    started = False
    reached_to = False
    total = 0.0

    for p in processes:  # processes are already ordered (first -> last)
        pid = str(p.process_id)

        if not started:
            if pid != from_pid:
                continue
            started = True

        if pid == from_pid and not include_from:
            pass
        else:
            total += float(p.finishing_price if use_finishing else p.scrap_price)

        if pid == to_pid:
            reached_to = True
            break

    if not started or not reached_to:
        return 0.0
    return max(0.0, total)

def Cum(pid: str, scrap_type: str = "scrap") -> float:
    if not processes:
        return 0.0
    return CumRange(str(processes[0].process_id), pid, scrap_type=scrap_type, include_from=True)

def ProcessPrice(pid: str, scrap_type: str = "scrap") -> float:
    p = process_by_id.get(str(pid))
    if not p:
        return 0.0
    return float(p.finishing_price if str(scrap_type).lower() == "finishing" else p.scrap_price)

def RiskPrice(r: Risk) -> float:
    return float(sum(max(0.0, float(a.cost)) for a in (r.preventive_actions + r.detection_actions)))



# Scenario calculations by steps

Step 1 (Occurrence):
- for non-finishing scraps: priceOfSingleScrap = Cum(cause)
- for finishing scraps: priceOfSingleScrap = process(cause).finishing
- nScrapsSavedByO = nScraps x reduction_O
- O_savings = nScrapsSavedByO x priceOfSingleScrap x 2

Step 2 (Detection):
- for non-finishing scraps: priceOfSingleScrapDetection = Cum(cause, consequence)
- for finishing scraps: priceOfSingleScrapDetection = process(cause).finishing
- nScrapsSavedByD = nScrapsForDetection x reduction_D
- D_savings = nScrapsSavedByD x priceOfSingleScrapDetection

Step 3:
- Net = sum(O_savings) + sum(D_savings) - risk_price



# Scenario 1 - Banduka

## Step 1: Occurrence (O)
- non-finishing: `priceOfSingleScrap = Cum(cause)`
- finishing: `priceOfSingleScrap = process(cause).finishing`
- `nScrapsSavedByO = nScraps x reduction_O`
- `O_savings = nScrapsSavedByO x priceOfSingleScrap x 2`

## Step 2: Detection (D)
- non-finishing: `priceOfSingleScrapDetection = Cum(cause, consequence)`
- finishing: `priceOfSingleScrapDetection = process(cause).finishing`
- `nScrapsForDetection = nScraps`
- `nScrapsSavedByD = nScrapsForDetection x reduction_D`
- `D_savings = nScrapsSavedByD x priceOfSingleScrapDetection`

## Step 3: Net
- `net_savings = O_savings_total + D_savings_total - risk_price`



In [38]:
cause_pid = str(risk_process_dd.value or risk.process_id)
relevant_rows = [s for s in scraps if str(s.cause_process_id) == cause_pid]

_banduka_step1_rows = []
_banduka_step2_rows = []

for s in relevant_rows:
    nScraps = float(max(0, int(s.count)))
    consequence_pid = str(s.consequence_process_id)
    scrap_type = str(s.scrap_type)
    isFinishingScrap = scrap_type.lower() == "finishing"

    # Step 1 O_savings
    # non-finishing: priceOfSingleScrap = Cum(cause)
    # finishing: priceOfSingleScrap = process(cause).finishing
    if isFinishingScrap:
        priceOfSingleScrap = ProcessPrice(cause_pid, scrap_type="finishing")
    else:
        priceOfSingleScrap = Cum(cause_pid, scrap_type=scrap_type)

    nScrapsSavedByO = nScraps * reduction_O
    O_savings = nScrapsSavedByO * priceOfSingleScrap * 2.0

    _banduka_step1_rows.append({
        "scrap_id": int(s.scrap_id),
        "scrap_type": scrap_type,
        "cause": cause_pid,
        "consequence": consequence_pid,
        "nScraps": nScraps,
        "reduction_O": reduction_O,
        "nScrapsSavedByO": nScrapsSavedByO,
        "priceOfSingleScrap": priceOfSingleScrap,
        "O_savings": O_savings,
    })

    # Step 2 D_savings
    # non-finishing: priceOfSingleScrapDetection = Cum(cause, consequence)
    # finishing: priceOfSingleScrapDetection = process(cause).finishing
    if isFinishingScrap:
        priceOfSingleScrapDetection = ProcessPrice(cause_pid, scrap_type="finishing")
    else:
        priceOfSingleScrapDetection = CumRange(cause_pid, consequence_pid, scrap_type=scrap_type, include_from=True)

    nScrapsForDetection = nScraps
    nScrapsSavedByD = nScrapsForDetection * reduction_D
    D_savings = nScrapsSavedByD * priceOfSingleScrapDetection

    _banduka_step2_rows.append({
        "scrap_id": int(s.scrap_id),
        "scrap_type": scrap_type,
        "cause": cause_pid,
        "consequence": consequence_pid,
        "nScrapsForDetection": nScrapsForDetection,
        "reduction_D": reduction_D,
        "nScrapsSavedByD": nScrapsSavedByD,
        "priceOfSingleScrapDetection": priceOfSingleScrapDetection,
        "D_savings": D_savings,
    })

banduka_step1_df = pd.DataFrame(_banduka_step1_rows)
banduka_step2_df = pd.DataFrame(_banduka_step2_rows)

display(widgets.HTML('<h4>Banduka - Step 1: Occurrence (O)</h4>'))
display(widgets.HTML('<div>non-finishing: priceOfSingleScrap = Cum(cause)<br/>finishing: priceOfSingleScrap = process(cause).finishing<br/>nScrapsSavedByO = nScraps x reduction_O<br/>O_savings = nScrapsSavedByO x priceOfSingleScrap x 2</div>'))
display(banduka_step1_df)

display(widgets.HTML('<h4>Banduka - Step 2: Detection (D)</h4>'))
display(widgets.HTML('<div>non-finishing: priceOfSingleScrapDetection = Cum(cause,consequence)<br/>finishing: priceOfSingleScrapDetection = process(cause).finishing<br/>nScrapsSavedByD = nScrapsForDetection x reduction_D<br/>D_savings = nScrapsSavedByD x priceOfSingleScrapDetection</div>'))
display(banduka_step2_df)

# Step 3 calculation:
# net_savings = O_savings_total + D_savings_total - risk_price
banduka_O_total = float(banduka_step1_df["O_savings"].sum()) if not banduka_step1_df.empty else 0.0
banduka_D_total = float(banduka_step2_df["D_savings"].sum()) if not banduka_step2_df.empty else 0.0
banduka_risk_price = RiskPrice(risk)
banduka_net = banduka_O_total + banduka_D_total - banduka_risk_price

banduka_step3_df = pd.DataFrame([{
    "scenario": "Banduka",
    "O_savings_total": banduka_O_total,
    "D_savings_total": banduka_D_total,
    "risk_price": banduka_risk_price,
    "net_savings": banduka_net,
}])

display(widgets.HTML('<h4>Banduka - Step 3: O + D - Risk Price</h4>'))
display(widgets.HTML('<div>net_savings = O_savings_total + D_savings_total - risk_price</div>'))
display(banduka_step3_df)




HTML(value='<h4>Banduka - Step 1: Occurrence (O)</h4>')

HTML(value='<div>non-finishing: priceOfSingleScrap = Cum(cause)<br/>finishing: priceOfSingleScrap = process(ca…

,scrap_id,scrap_type,cause,consequence,nScraps,reduction_O,nScrapsSavedByO,priceOfSingleScrap,O_savings
0,1,scrap,P3,P5,100.0,0.5,50.0,36.0,3600.0
1,2,scrap,P3,P7,30.0,0.5,15.0,36.0,1080.0
2,3,finishing,P3,P5,30.0,0.5,15.0,9.0,270.0
3,4,finishing,P3,P7,30.0,0.5,15.0,9.0,270.0


HTML(value='<h4>Banduka - Step 2: Detection (D)</h4>')

HTML(value='<div>non-finishing: priceOfSingleScrapDetection = Cum(cause,consequence)<br/>finishing: priceOfSin…

,scrap_id,scrap_type,cause,consequence,nScrapsForDetection,reduction_D,nScrapsSavedByD,priceOfSingleScrapDetection,D_savings
0,1,scrap,P3,P5,100.0,0.5,50.0,48.0,2400.0
1,2,scrap,P3,P7,30.0,0.5,15.0,90.0,1350.0
2,3,finishing,P3,P5,30.0,0.5,15.0,9.0,135.0
3,4,finishing,P3,P7,30.0,0.5,15.0,9.0,135.0


HTML(value='<h4>Banduka - Step 3: O + D - Risk Price</h4>')

HTML(value='<div>net_savings = O_savings_total + D_savings_total - risk_price</div>')

,scenario,O_savings_total,D_savings_total,risk_price,net_savings
0,Banduka,5220.0,4020.0,2000.0,7240.0


# Scenario 2 - Andrija

## Step 1: Occurrence (O)
- non-finishing: `priceOfSingleScrap = Cum(cause)`
- finishing: `priceOfSingleScrap = process(cause).finishing`
- `nScrapsSavedByO = nScraps x reduction_O`
- `O_savings = nScrapsSavedByO x priceOfSingleScrap x 2`

## Step 2: Detection (D)
- non-finishing: `priceOfSingleScrapDetection = Cum(cause, consequence)`
- finishing: `priceOfSingleScrapDetection = process(cause).finishing`
- `nScrapsForDetection = nScraps - nScrapsSavedByO`
- `nScrapsSavedByD = nScrapsForDetection x reduction_D`
- `D_savings = nScrapsSavedByD x priceOfSingleScrapDetection`

## Step 3: Net
- `net_savings = O_savings_total + D_savings_total - risk_price`



In [39]:
# -------------------------
# Scenario 2 - Andrija (3-step table)
# D step uses remaining scraps after O savings.
# -------------------------

# Step 1 calculation:
# non-finishing: priceOfSingleScrap = Cum(cause)
# finishing: priceOfSingleScrap = process(cause).finishing
# nScrapsSavedByO = nScraps x reduction_O
# O_savings = nScrapsSavedByO x priceOfSingleScrap x 2

cause_pid = str(risk_process_dd.value or risk.process_id)
relevant_rows = [s for s in scraps if str(s.cause_process_id) == cause_pid]

_andrija_step1_rows = []
_andrija_step2_rows = []

for s in relevant_rows:
    nScraps = float(max(0, int(s.count)))
    consequence_pid = str(s.consequence_process_id)
    scrap_type = str(s.scrap_type)
    isFinishingScrap = scrap_type.lower() == "finishing"

    # Step 1: O savings
    if isFinishingScrap:
        priceOfSingleScrap = ProcessPrice(cause_pid, scrap_type="finishing")
    else:
        priceOfSingleScrap = Cum(cause_pid, scrap_type=scrap_type)

    nScrapsSavedByO = nScraps * reduction_O
    O_savings = nScrapsSavedByO * priceOfSingleScrap * 2.0

    _andrija_step1_rows.append({
        "scrap_id": int(s.scrap_id),
        "scrap_type": scrap_type,
        "cause": cause_pid,
        "consequence": consequence_pid,
        "nScraps": nScraps,
        "reduction_O": reduction_O,
        "nScrapsSavedByO": nScrapsSavedByO,
        "priceOfSingleScrap": priceOfSingleScrap,
        "O_savings": O_savings,
    })

    # Step 2 calculation:
    # non-finishing: priceOfSingleScrapDetection = Cum(cause, consequence)
    # finishing: priceOfSingleScrapDetection = process(cause).finishing
    # nScrapsForDetection = nScraps - nScrapsSavedByO
    # nScrapsSavedByD = nScrapsForDetection x reduction_D
    # D_savings = nScrapsSavedByD x priceOfSingleScrapDetection
    if isFinishingScrap:
        priceOfSingleScrapDetection = ProcessPrice(cause_pid, scrap_type="finishing")
    else:
        priceOfSingleScrapDetection = CumRange(cause_pid, consequence_pid, scrap_type=scrap_type, include_from=True)

    nScrapsForDetection = max(0.0, nScraps - nScrapsSavedByO)
    nScrapsSavedByD = nScrapsForDetection * reduction_D
    D_savings = nScrapsSavedByD * priceOfSingleScrapDetection

    _andrija_step2_rows.append({
        "scrap_id": int(s.scrap_id),
        "scrap_type": scrap_type,
        "cause": cause_pid,
        "consequence": consequence_pid,
        "nScrapsForDetection": nScrapsForDetection,
        "reduction_D": reduction_D,
        "nScrapsSavedByD": nScrapsSavedByD,
        "priceOfSingleScrapDetection": priceOfSingleScrapDetection,
        "D_savings": D_savings,
    })

andrija_step1_df = pd.DataFrame(_andrija_step1_rows)
andrija_step2_df = pd.DataFrame(_andrija_step2_rows)

display(widgets.HTML('<h4>Andrija - Step 1: Occurrence (O)</h4>'))
display(widgets.HTML('<div>non-finishing: priceOfSingleScrap = Cum(cause)<br/>finishing: priceOfSingleScrap = process(cause).finishing<br/>nScrapsSavedByO = nScraps x reduction_O<br/>O_savings = nScrapsSavedByO x priceOfSingleScrap x 2</div>'))
display(andrija_step1_df)

display(widgets.HTML('<h4>Andrija - Step 2: Detection (D)</h4>'))
display(widgets.HTML('<div>non-finishing: priceOfSingleScrapDetection = Cum(cause,consequence)<br/>finishing: priceOfSingleScrapDetection = process(cause).finishing<br/>nScrapsForDetection = nScraps - nScrapsSavedByO<br/>nScrapsSavedByD = nScrapsForDetection x reduction_D<br/>D_savings = nScrapsSavedByD x priceOfSingleScrapDetection</div>'))
display(andrija_step2_df)

# Step 3 calculation:
# net_savings = O_savings_total + D_savings_total - risk_price
andrija_O_total = float(andrija_step1_df["O_savings"].sum()) if not andrija_step1_df.empty else 0.0
andrija_D_total = float(andrija_step2_df["D_savings"].sum()) if not andrija_step2_df.empty else 0.0
andrija_risk_price = RiskPrice(risk)
andrija_net = andrija_O_total + andrija_D_total - andrija_risk_price

andrija_step3_df = pd.DataFrame([{
    "scenario": "Andrija",
    "O_savings_total": andrija_O_total,
    "D_savings_total": andrija_D_total,
    "risk_price": andrija_risk_price,
    "net_savings": andrija_net,
}])

display(widgets.HTML('<h4>Andrija - Step 3: O + D - Risk Price</h4>'))
display(widgets.HTML('<div>net_savings = O_savings_total + D_savings_total - risk_price</div>'))
display(andrija_step3_df)




HTML(value='<h4>Andrija - Step 1: Occurrence (O)</h4>')

HTML(value='<div>non-finishing: priceOfSingleScrap = Cum(cause)<br/>finishing: priceOfSingleScrap = process(ca…

,scrap_id,scrap_type,cause,consequence,nScraps,reduction_O,nScrapsSavedByO,priceOfSingleScrap,O_savings
0,1,scrap,P3,P5,100.0,0.5,50.0,36.0,3600.0
1,2,scrap,P3,P7,30.0,0.5,15.0,36.0,1080.0
2,3,finishing,P3,P5,30.0,0.5,15.0,9.0,270.0
3,4,finishing,P3,P7,30.0,0.5,15.0,9.0,270.0


HTML(value='<h4>Andrija - Step 2: Detection (D)</h4>')

HTML(value='<div>non-finishing: priceOfSingleScrapDetection = Cum(cause,consequence)<br/>finishing: priceOfSin…

,scrap_id,scrap_type,cause,consequence,nScrapsForDetection,reduction_D,nScrapsSavedByD,priceOfSingleScrapDetection,D_savings
0,1,scrap,P3,P5,50.0,0.5,25.0,48.0,1200.0
1,2,scrap,P3,P7,15.0,0.5,7.5,90.0,675.0
2,3,finishing,P3,P5,15.0,0.5,7.5,9.0,67.5
3,4,finishing,P3,P7,15.0,0.5,7.5,9.0,67.5


HTML(value='<h4>Andrija - Step 3: O + D - Risk Price</h4>')

HTML(value='<div>net_savings = O_savings_total + D_savings_total - risk_price</div>')

,scenario,O_savings_total,D_savings_total,risk_price,net_savings
0,Andrija,5220.0,2010.0,2000.0,5230.0


In [40]:
# -------------------------
# Scenario comparison (Step 3 totals)
# -------------------------

_summary_rows = []
if 'banduka_step3_df' in globals() and not banduka_step3_df.empty:
    _summary_rows.append(dict(banduka_step3_df.iloc[0]))
if 'andrija_step3_df' in globals() and not andrija_step3_df.empty:
    _summary_rows.append(dict(andrija_step3_df.iloc[0]))

comparison_df = pd.DataFrame(_summary_rows)
display(comparison_df)



,scenario,O_savings_total,D_savings_total,risk_price,net_savings
0,Banduka,5220.0,4020.0,2000.0,7240.0
1,Andrija,5220.0,2010.0,2000.0,5230.0
